In [1]:
import numpy as np
import os
import os.path as op
import h5py
import shutil
import requests
import statistics

In [2]:
outpath = '/Users/aleex/DSE/Machine Learning/Project/label/'

In [62]:
def createData(outpath):
    # PATH TO STORE THE TRAINING SAMPLES
    #outpath = '/Users/aleex/DSE/Machine Learning/Project/label/'
    if not os.path.exists(outpath):
        os.makedirs(outpath)

    def genLabel(goes16, goes17):
        n1 = len(goes16)*len(goes16[0])
        n2 = len(goes17)*len(goes17[0])

        std1 = np.std(goes16)
        std2 = np.std(goes17)

        diff = goes16 - goes17
        diff_std = np.std(diff)

        delta = abs(goes16 - goes17)
        label = delta//diff_std
        label = label.astype(int)

        return label

    def writeH5(data, label, uniTime):
        coords = np.array(data).astype(np.float)
        shape = coords.shape
        hdf5_path = outpath + uniTime + '.h5'
        with h5py.File(hdf5_path, mode='w') as f:
            d = f.create_dataset('/data', data = coords)
            l = f.create_dataset('/label', data = label)

    #ROOT URL OF THE NPY FILES
    rootURL = "https://gitlab.com/adomakor412/goesdata/-/raw/master/spring_npy/"
    for idx in range(98, 109):    
        for h in range(0, 24):
            for m in range(0, 6):
                data = np.empty((401,1001,0), float)
                label = np.empty((401,1001,0), int)
                for band in range(7, 11):
                    with open('log2', 'a') as log:
                        timeID = f"s2019{str(idx).zfill(3)}{str(h).zfill(2)}{str(m*10).zfill(2)}"

                        goes16_url = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C{str(band).zfill(2)}_G16_{timeID}"
                        goes17_url = rootURL + f"{str(idx).zfill(3)}/OR_ABI-L1b-RadF-M6C{str(band).zfill(2)}_G17_{timeID}"
                        
                        try:
                            response16 = requests.get(goes16_url, stream=True)
                            response17 = requests.get(goes17_url, stream=True)

                            print(timeID)

                            with open('temp16.npy', 'wb') as temp16:
                                shutil.copyfileobj(response16.raw, temp16)
                            with open('temp17.npy', 'wb') as temp17:
                                shutil.copyfileobj(response17.raw, temp17)               

                            data16 = np.load('temp16.npy').reshape((401, 1001, 1))
                            data17 = np.load('temp17.npy').reshape((401, 1001, 1))
                            
                            labels = genLabel(data16, data17)
                            
                            data = np.append(data, data17, axis = 2)
                            #print(data)
                            
                            label = np.append(label, labels, axis = 2)
                            
                            #vote = np.round(np.median(label, axis=2))
                            #writeH5(stack, vote, timeID)

                        except ValueError as e:
                            print("url doesn't exist for " + str(goes16_url))
                            print(e)
                            print('\n')
                            print(str(goes16_url), file=log)
                            print('\n')
                            print(e, file=log)
                            break

                data = data.reshape((401*1001, -1))
                vote = np.round(np.median(label, axis=2)).astype('int')                
                vote = vote.reshape((-1))
                writeH5(data, vote, timeID)

In [61]:
createData(outpath)

s20190980000
s20190980000
s20190980000
s20190980000
[[ 0.82905048  4.54261971 12.72240067 20.98910141]
 [ 0.83687222  4.54261971 12.72240067 20.94901848]
 [ 0.82905048  4.54972458 12.72240067 20.98910141]
 ...
 [ 0.86190188  3.88187695 10.89673328 18.40377617]
 [ 0.8603375   3.83924818 10.85165501 18.42381668]
 [ 0.86190188  3.86056232 10.87419415 18.42381668]]
[1 1 0 ... 0 0 0]
s20190980010
s20190980010
s20190980010
s20190980010
[[ 0.82435745  4.52130556 12.69986153 21.04922485]
 [ 0.81497133  4.54261971 12.65478325 20.98910141]
 [ 0.80871391  4.52130556 12.63224411 21.00914192]
 ...
 [ 0.85877317  3.84635305 10.85165501 18.46389961]
 [ 0.85720879  3.81793404 10.82911587 18.40377617]
 [ 0.85408008  3.83924818 10.82911587 18.38373375]]
[1 1 0 ... 0 0 0]
s20190980020
s20190980020
s20190980020


KeyboardInterrupt: 